# 0. Installing requirements, loading packages & modules

In [1]:
# creating virtual environment
!python -m venv ENV 

#!pip list
# install module requirements
!pip install -r requirements.txt

# loading packages 
import csv
import requests
import json
from bs4 import BeautifulSoup
from datetime import datetime
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait as WBW
from selenium.webdriver.support import expected_conditions as EC

# Loading module with scraping functions
from scraper import *

  Using cached selenium-2.16.0.tar.gz (3.9 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      no previously-included directories found matching '*'
      Traceback (most recent call last):
        File "<string>", line 2, in <module>
        File "<pip-setuptools-caller>", line 34, in <module>
        File "C:\Users\leona\AppData\Local\Temp\pip-install-1swm9se1\selenium_7dd32c159e8b40efb0a6321e18ef5376\setup.py", line 55, in <module>
          src_root = setup_python3()
                     ^^^^^^^^^^^^^^^
        File "C:\Users\leona\AppData\Local\Temp\pip-install-1swm9se1\selenium_7dd32c159e8b40efb0a6321e18ef5376\setup.py", line 47, in setup_python3
          util.run_2to3(outfiles_2to3)
          ^^^^^^^^^^^^^
      AttributeError: module 'distutils.util' has no attribute 'run_2to3'
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

× En

# 1. Scraping URLS of Phone Products

#### DatArt

In [2]:
# 1 Finding the basic URL that can be extended to obtain the links of certain products
DatArt_basic_url = "https://www.datart.cz"

# 2 Finding category website with all the products listed (here by manually setting the limit of products shown manually in the browser)
DataArt_category_url = "https://www.datart.cz/mobilni-telefony.html?limit=1100" 

product_urls_DatArt = get_product_urls_DatArt(DatArt_basic_url, DataArt_category_url)

933 products found.(2024-02-04 20:24:36)


#### CZC

In [3]:
# 1 Finding the basic URL that can be extended to obtain the links of certain products
CZC_basic_url = "https://www.czc.cz"

# 2 Finding category website with all the products listed (here no manual limit for the number of products shown can be set)
CZC_category_url = "https://www.czc.cz/mobilni-telefony/produkty"

# 3 Instead all pages of the category url have to be individually scraped
CZC_category_urls = [CZC_category_url] # category_url is added as it is the first page with products

# 4 Generating URL for all pages of the category website

# Create list of page limits
page_limits = list(range(27, 1000, 27)) 

# Iterate over the pages and construct URLs for every page of the category website
for page_limit in page_limits:
    url_with_page_limit = f"{CZC_category_url}?q-first={page_limit}"
    CZC_category_urls.append(url_with_page_limit)

product_urls_CZC = get_product_urls_CZC(CZC_basic_url, CZC_category_urls)

Page 1 complete
Page 2 complete
Page 3 complete
Page 4 complete
Page 5 complete
Page 6 complete
Page 7 complete
Page 8 complete
Page 9 complete
Page 10 complete
Page 11 complete
Page 12 complete
Page 13 complete
Page 14 complete
Page 15 complete
Page 16 complete
Page 17 complete
Page 18 complete
Page 19 complete
Page 20 complete
Page 21 complete
Page 22 complete
Page 23 complete
Page 24 complete
Page 25 complete
Page 26 complete
Page 27 complete
Page 28 complete
Page 29 complete
Page 30 complete
Page 31 complete
Page 32 complete
Page 33 complete
Page 34 complete
Page 35 complete
Page 36 complete
Page 37 complete
Page 38 complete
857 products found.(2024-01-24 12:04:57)


#### Electroworld

In [3]:
# 1 Finding the basic URL that can be extended to obtain the links of certain products
Electroworld_basic_url = "https://www.electroworld.cz"

# 2 Finding category page with all the products listed (here no manual limit for the number of products shown can be set)
Electroworld_category_url =  "https://www.electroworld.cz/chytre-mobily/sort-by_mostExpensive" # sorted by most expensive gives unique products for each page

product_urls_Electroworld = get_product_urls_Electroworld(Electroworld_basic_url, Electroworld_category_url, 50) # Currently (04.02.2024) there are 41 pages of products

Page 1 complete
Page 2 complete
Page 3 complete
Page 4 complete
Page 5 complete
Page 6 complete
Page 7 complete
Page 8 complete
Page 9 complete
Page 10 complete
Page 11 complete
Page 12 complete
Page 13 complete
Page 14 complete
Page 15 complete
Page 16 complete
Page 17 complete
Page 18 complete
Page 19 complete
Page 20 complete
Page 21 complete
Page 22 complete
Page 23 complete
Page 24 complete
Page 25 complete
Page 26 complete
Page 27 complete
Page 28 complete
Page 29 complete
Page 30 complete
Page 31 complete
Page 32 complete
Page 33 complete
Page 34 complete
Page 35 complete
Page 36 complete
Page 37 complete
Page 38 complete
Page 39 complete
Page 40 complete
Page 41 complete
Page 42 complete
Page 43 complete
Page 44 complete
Page 45 complete
Page 46 complete
Page 47 complete
Page 48 complete
Page 49 complete
737 products found.(2024-02-04 20:26:21)


# 2. Scraping Product Information 

#### DatArt

For all products (product URL's) found, the product information is scraped. For every characteristic, a function is defined in scraper.py. For some variables like wireless technologies, lists of characteristics are scraped. For each of these characteristics a 1-values is assigned to show that these characteristic is available.

In [4]:
products_DatArt = get_product_info_DatArt(product_urls_DatArt)

Product 1 finished
Product 2 finished
Product 3 finished
Product 4 finished
Product 5 finished
Product 6 finished
Product 7 finished
Product 8 finished
Product 9 finished
Product 10 finished
Product 11 finished
Product 12 finished
Product 13 finished
Product 14 finished
Product 15 finished
Product 16 finished
Product 17 finished
Product 18 finished
Product 19 finished
Product 20 finished
Product 21 finished
Product 22 finished
Product 23 finished
Product 24 finished
Product 25 finished
Product 26 finished
Product 27 finished
Product 28 finished
Product 29 finished
Product 30 finished
Product 31 finished
Product 32 finished
Product 33 finished
Product 34 finished
Product 35 finished
Product 36 finished
Product 37 finished
Product 38 finished
Product 39 finished
Product 40 finished
Product 41 finished
Product 42 finished
Product 43 finished
Product 44 finished
Product 45 finished
Product 46 finished
Product 47 finished
Product 48 finished
Product 49 finished
Product 50 finished
Product 5

Post-processing Data. Adding zero values for binary variables with missing values. After that the variables data type is changed to boolean.

In [9]:
# Keys to ensure they exist in each dictionary with a default value of 0
keys_binary = ['without fingerprint reader', 'Bokeh effect', '4G/LTE', 'Galileo', 'Beidou', 'auto focus', 'thermal camera', 'GPS', 'body fingerprint reader', 'removable battery', 'infrared', 'fast charging', 'wide angle lens', 'Wi-Fi', 'memory card slot', 'macro mode', 'night mode', 'FM-radio', 'optical zoom', '5G', 'wireless charging', 'face unlock', '3.5mm jack', 'NFC', 'GLONASS', 'telephoto', 'A-GPS', 'reverse wireless charging', 'in-display fingerprint reader', 'Bluetooth', 'flash diode', 'hybrid zoom']

# Iterate through each dictionary and set missing values of binary variables to zero
for product in products_DatArt:
    # Add missing keys with a default value of 0
    for key in keys_binary:
        product.setdefault(key, 0)

# Converting binary 1/0 variables to boolean data type
for product in products_DatArt:
    for variable in keys_binary:
        if variable in product:
            product[variable] = bool(product[variable])


For some products the product width and length was confused on the product page. Therefore, for products with length <8 cm, the length and the width are interchanged. After a manual check, this boundary is the highest appropriate to include all products that have a small product length.

In [ ]:
for product in products_DatArt:
    if product['length'] < 8:
        length_val = product['length']
        width_val = product['width']
        product['length'] = width_val
        product['width'] = length_val

Converting the list of dictionaries into a json file "product_info_DatArt.json".

In [2]:
# convert to JSON object and safe as JSON file
with open('product_info_DatArt.json', 'w') as file:
    json.dump(products_DatArt, file)

NameError: name 'products_DatArt' is not defined

Saving the data also as a csv file.

In [12]:
# Specify the paths for CSV files
csv_file_path = 'Product_Table_DatArt.csv'

# Open the JSON file and load its content
with open('product_info_DatArt.json', 'r') as json_file:
    Products_Data_DatArt = json.load(json_file)
    
# Extract all unique keys from the JSON data
all_keys = set()
for entry in Products_Data_DatArt:
    all_keys.update(entry.keys())

# Move "ID" to the front of the list of keys
all_keys = ['ID'] + ['title'] + list(all_keys - {'ID'}- {'title'})

# Open the CSV file for writing
with open(csv_file_path, 'w', newline='') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)

    # Write the header
    csv_writer.writerow(all_keys)

    # Write the data
    for entry in Products_Data_DatArt:
        # Create a row with "ID" as the first column
        row_data = [entry.get('ID', '')] + [entry.get('title', '')] + [entry.get(key, '') for key in all_keys[2:]]
        csv_writer.writerow(row_data)

#### Electroworld 

For the products (product URL's) from Electroworld, only the BeatifulSoup object is not enough to scrape the product characteristics. Here, a large part of the product characteristics is hidden behind a button. Therefore, the selenium module is used to click on the button in the background and scrape the data from the resulting dynamic webpage.

In [13]:
products_Electroworld = get_product_info_Electroworld(product_urls_Electroworld)

Attempt 1 failed. Exception: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7B1165E42+3538674]
	(No symbol) [0x00007FF7B0D84C02]
	(No symbol) [0x00007FF7B0C35AEB]
	(No symbol) [0x00007FF7B0C7BF4E]
	(No symbol) [0x00007FF7B0C7C0CC]
	(No symbol) [0x00007FF7B0CBE477]
	(No symbol) [0x00007FF7B0C9F0EF]
	(No symbol) [0x00007FF7B0CBBDE3]
	(No symbol) [0x00007FF7B0C9EE53]
	(No symbol) [0x00007FF7B0C6F514]
	(No symbol) [0x00007FF7B0C70631]
	GetHandleVerifier [0x00007FF7B1196CAD+3738973]
	GetHandleVerifier [0x00007FF7B11EC506+4089270]
	GetHandleVerifier [0x00007FF7B11E4823+4057299]
	GetHandleVerifier [0x00007FF7B0EB5C49+720121]
	(No symbol) [0x00007FF7B0D9126F]
	(No symbol) [0x00007FF7B0D8C304]
	(No symbol) [0x00007FF7B0D8C432]
	(No symbol) [0x00007FF7B0D7BD04]
	BaseThreadInitThunk [0x00007FF952BD257D+29]
	RtlUserThreadStart [0x00007FF953FEAA58+40]

Attempt 2 failed. Exception: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF7B1165E42+3538674]
	(No symbol) [0x00007FF7B0D84C02]
	(No symbol

Post-Processing. Adding zeros to missing values of binary variables. After that, converting these into boolean data type.

In [14]:
keys_binary = ['face unlock', 'body fingerprint reader', 'in-display fingerprint reader']

# Iterate through each dictionary
for product in products_Electroworld:
    # Add missing keys with a default value of 0
    for key in keys_binary:
        product.setdefault(key, 0)

# Converting binary 1/0 variables to boolean data type
for product in products_Electroworld:
    for variable in keys_binary:
        if variable in product:
            product[variable] = bool(product[variable])

For some products the product depth and length was confused on the product page. Therefore, for products with length <2 cm, the length and the depth are interchanged.

In [ ]:
for product in products_Electroworld:
    if product['length'] < 8:
        length_val = product['length']
        depth_val = product['depth']
        product['length'] = depth_val
        product['depth'] = length_val

Converting the list of dictionaries into a json file "product_info_Electroworld.json".

In [15]:
# convert to JSON object and safe as JSON file
with open('product_info_Electroworld.json', 'w') as file:

    json.dump(products_Electroworld, file)

In [16]:
# Specify the paths for CSV files
csv_file_path = 'Product_Table_Electroworld.csv'

# Open the JSON file and load its content
with open('product_info_Electroworld.json', 'r') as json_file:
    Products_Data_Electroworld = json.load(json_file)
    
# Extract all unique keys from the JSON data
all_keys = set()
for entry in Products_Data_Electroworld:
    all_keys.update(entry.keys())

# Move "ID" and "title" to the front of the list of keys
all_keys = ['ID'] + ['title'] + list(all_keys - {'ID'} - {'title'})

# Open the CSV file for writing
with open(csv_file_path, 'w', newline='') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)

    # Write the header
    csv_writer.writerow(all_keys)

    # Write the data
    for entry in Products_Data_Electroworld:
        # Create a row with "ID" as the first column
        row_data = [entry.get('ID', '')] + [entry.get('title', '')] + [entry.get(key, '') for key in all_keys[2:]]
        csv_writer.writerow(row_data)

## 3. Merging Data

In [5]:
with open('product_info_DatArt.json', 'r') as json_file:
    Merge_Products_Data_DatArt = json.load(json_file)

with open('product_info_Electroworld.json', 'r') as json_file:
    Merge_Products_Data_Electroworld = json.load(json_file)

# Get the common variables of two lists of product dictionaries (DatArt, Electroworld)
common_variables = get_common_variables(Merge_Products_Data_DatArt, Merge_Products_Data_Electroworld)

# Create a new list of dictionaries with only the specified variables from both data sources
complete_products_list = []
for product_dict in Merge_Products_Data_DatArt + Merge_Products_Data_Electroworld:
    complete_products_list.append(extract_variables(product_dict, common_variables))

# Add unique 'ID' to each product dictionary
id_counter = 1
for product_dict in complete_products_list:
    product_dict['ID'] = id_counter
    id_counter += 1

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [11]:
# convert to JSON object and safe as JSON file
with open('product_info_complete.json', 'w') as file:

    json.dump(complete_products_list, file)

# Specify the paths for CSV files
csv_file_path = 'product_table_complete.csv'

# Open the JSON file and load its content
with open('product_info_complete.json', 'r') as json_file:
    Products_Data_complete = json.load(json_file)
    
# Extract all unique keys from the JSON data
all_keys = set()
for entry in Products_Data_complete:
    all_keys.update(entry.keys())

# Move "ID" and "title" to the front of the list of keys
all_keys = ['ID'] + ['online-retailer'] + ['title'] + list(all_keys - {'ID'} - {'online-retailer'} - {'title'})

# Open the CSV file for writing
with open(csv_file_path, 'w', newline='') as csv_file:
    # Create a CSV writer object
    csv_writer = csv.writer(csv_file)

    # Write the header
    csv_writer.writerow(all_keys)

    # Write the data
    for entry in Products_Data_complete:
        # Create a row with "ID" as the first column
        row_data = [entry.get('ID', '')] + [entry.get('online-retailer', '')] + [entry.get('title', '')] + [entry.get(key, '') for key in all_keys[3:]]
        csv_writer.writerow(row_data)

Next steps: 
- Merging DataArt and Electroworld 
- Datatesting
- Data modelling
- runnability
- (For CZC/Electroworld code: Automatic handling of pages, instead of manually setting the project number limit of 1000 above. 27 can stay if necessary (or trying out numbers 1:50 until data is obtained))

- 1. Adding packages with init files and .py modules to wd for: testing, main module and create setup.py file
- 2. Data analysis
- 3. Runnability


#### 3. Ideas for Analysis

Steps:
0) Which data can we use legally without problems? Ebay, DataArt etc.

1) Creating a function to save all necessary data for one product simultaneously.
2) Create a function to let the function from 1) run over all product URL's.
2) (How to save the data of the product description -> we need to look at different products and see what variables we can define.)
3) How to work with github, commits, pull etc.?
4) How to create a nice table in github to document aims and checkpoints.
5) How to structure the repository -> documents for functions, loading packages, ReadMe etc.
6) Running scripts on christmas again, to see if their is a price drop because of price offers
7) Starting data analysis -> simple averages, counts etc. -> can we create a dashboard on github? (Share or products etc.)
8) Further Statistical Analysis - Ideas:
    1) Predicing prices for new products (price modelling), finding out which variables drive prices the most (factor)-> Does it makes sense to do a price model as prices consist largely of the purchase price; could we find those purchase prices somewhere and join them with our data?
    2) Brand Analysis: Explore the average prices of smartphones for each brand. Analyze market share based on the number of products or total sales for each brand.
    3) Storage Impact: Investigate how storage capacity (GB) affects smartphone prices. Explore the most popular storage capacity among consumers.
    4) Segment the market based on different characteristics (e.g., high-end, mid-range, budget). One could then do 1) on all different segments to explore differences.
    5) Consumer Preferences: Survey or analyze customer reviews to understand what features users value the most. Identify common positive and negative sentiments related to smartphone characteristics.
    6) Profitability Analysis (given that we know production or purchase prices): Estimate profit margins for different smartphones based on their production costs and prices.
Identify the most profitable products in your dataset. One could use the number of ratings as a proxy for the number of sales.


8) Which data tests and checks can we include?

How we could build a price model and find out which variables drive smartphone prices the most:

Feature Selection:

1) Use techniques like correlation analysis, mutual information, or feature importance from tree-based models to identify the most relevant features.
Consider removing highly correlated features to avoid multicollinearity.
Model Selection:

2) Choose a regression model that suits your data and problem. Common choices include linear regression, decision tree regression, random forest regression, or gradient boosting algorithms.
Model Training:

3) Split your dataset into training and testing sets.
Train your chosen model on the training data.
Model Evaluation:

4) Evaluate the model's performance on the testing set using appropriate metrics (e.g., mean squared error, R-squared, etc.).
Analyze the residuals to check for any patterns or systematic errors.

5) Iterate and Refine hyperparameters etc.